## Random Forest

**Random forests**, also known as **Random Decision Forests**, are an ensemble learning method for classification, regression, and other problems that works by training a large number of **decision trees**. For classification tasks, the random forest's output is the class chosen by the majority of trees. The mean or average prediction of the individual trees is returned for regression tasks. In this assignment we will be using Random Forest Classifier to predict the possibility of breast cancer being benign or not.

Note that the code for building a decision tree will be similar to your first assignment. Feel free to reuse the code from your first assignment but be ware of the differences.

In [ ]:
# Do not change anything in this cell.
import numpy as np
from collections import Counter
from sklearn import datasets
from sklearn.model_selection import train_test_split

We are using the breast cancer dataset provided in scikit-learn. Here is a look at your dataset.

In [ ]:
# Do not change anything in this cell.
data = datasets.load_breast_cancer()
X = data.data
y = data.target

In [ ]:
# Do not change anything in this cell.
data

{'DESCR': '.. _breast_cancer_dataset:\n\nBreast cancer wisconsin (diagnostic) dataset\n--------------------------------------------\n\n**Data Set Characteristics:**\n\n    :Number of Instances: 569\n\n    :Number of Attributes: 30 numeric, predictive attributes and the class\n\n    :Attribute Information:\n        - radius (mean of distances from center to points on the perimeter)\n        - texture (standard deviation of gray-scale values)\n        - perimeter\n        - area\n        - smoothness (local variation in radius lengths)\n        - compactness (perimeter^2 / area - 1.0)\n        - concavity (severity of concave portions of the contour)\n        - concave points (number of concave portions of the contour)\n        - symmetry \n        - fractal dimension ("coastline approximation" - 1)\n\n        The mean, standard error, and "worst" or largest (mean of the three\n        largest values) of these features were computed for each image,\n        resulting in 30 features.  For

In [ ]:
# Write a function to calculate the entropy. We will be using this to calculate
## the information gain for decision tree.
def entropy(y):
    # Write your code here
    class_labels = np.unique(y)
    # Initialize the entropy
    entropy = 0
    # Calculate the entropy
    for cls in class_labels:
        p_cls = len(y[y == cls]) / len(y)
        entropy += (-p_cls * np.log2(p_cls))

    return entropy
    return None

In [ ]:
# Do not change anything in this cell.
class Node:

    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [ ]:
# Write code in this cell to build your Decision Trees used for Random Forest Classifier
#This will be similar to assignment 1, but keep in mind that we will randomly pick 
#a subset of features for splitting, at then choose the best variable/split-point among those
class DecisionTree:

    def __init__(self, min_samples_split=2, max_depth=100, num_features=None):
        # Initialize the min_split,max_depth, root and num_features are set to None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_features = num_features
        self.root = None

    # Function to train the decision tree
    def fit(self, X, y):
        
        """
        Function to train the tree.
        X: Features
        Y: Target
        """
        self.num_features = X.shape[1] if not self.num_features else min(self.num_features, X.shape[1])
        self.root = self.build_tree(X, y)

    # Write a function to predict the results for a given dataset, using the tree you built 
    #and the tree traversal function
    def predict(self, X):
        """
        Prediction function to calculate the all the predictions of the matrix of features 
        provided using make_predictions function
        X: Matrix of features
        Returns predictions
        """
        return self.tree_traversal(X, self.root)

    # build the decision tree
    def build_tree(self, X, y, depth=0):
        """
        This will be a recursive function to build the decision tree.
        X: The data that you will be using for your assignment
        y: Target
        depth: Current depth of the tree
        Returns the leaf node
        """
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping criteria should be when depth equals or exceeds max depth
        # or there is only one label at the node
        # or when n_samples gets smaller than min_samples_split
        if (depth > self.max_depth or n_labels == 1 or n_samples <= self.min_samples_split):
            leaf_value = self.max_frequency_label(y)
            return Node(value=leaf_value)
        # select the features **randomly**. Hint: you can use functions like np.random.choice
        feature_idxs = np.random.choice(n_features, int(n_features / 10))

        # find the best split according to information gain
        best_feat, best_thresh = self.optimal_criterion(X, y, feature_idxs)
      
        # Take the result from the split and recuisively grow the tree
        left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
        left = self.build_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self.build_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    def optimal_criterion(self, X, y, feature_idxs):
        """
        Find the optimal criterion for the split of the tree, using the selected features.
        X: dataset
        y: target
        feature_idxs: randomly selected feature idxs
        Return split index and threshold
        """
        # initialize the best gain
        best_gain = -1
        for feature_index in feature_idxs:
            # Find unique threshold values
            possible_thresholds = np.unique(X[:, feature_index])
            for threshold in possible_thresholds:
                #dataset_left, dataset_right = self._split(X[:, feature_index], threshold)
                curr_info_gain = self._information_gain(y, X[:, feature_index], threshold)
                if curr_info_gain > best_gain:
                    best_gain = curr_info_gain
                    best_feature_index = feature_index
                    best_thres = threshold

        # Best feature among the features
        split_idx = best_feature_index
        # Best threshold for the best feature
        split_thresh = best_thres
        return split_idx, split_thresh

    def _information_gain(self, y, X_column, split_thresh):
        """
        Function to return the information gain
        y: target
        X: data
        split_thresh: threshold for split
        Return Information gain
        """
        # parent loss
        parent_entropy = entropy(y)

        # generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh)

        entropy_left = entropy(y[left_idxs]) * len(left_idxs) / len(y)
        entropy_right = entropy(y[right_idxs]) * len(right_idxs) / len(y)
        information_gain = parent_entropy - entropy_left - entropy_right
        return information_gain

    def _split(self, X_column, split_thresh):
        """
          (Already implemented)
          The split function 
          X_column: data
          split_thresh: threshold value
          Return left_idxs, right_idxs
          """
        # Function to split the tree
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs
  
    def tree_traversal(self, x, node):
        """
          Tree traversal method which returns (one of) the leaf node value
          x: data
          node: node of the tree
          """
        if node.is_leaf_node():
            return node.value
  
        if x[node.feature] <= node.threshold:
            return self.tree_traversal(x, node.left)
        return self.tree_traversal(x, node.right)

    def max_frequency_label(self, y):
        """
          Determine the target label with maximum frequency. Hint: You can use collections.Counter()
          y: target
          """
        most_common = np.argmax(np.bincount(y))
        return most_common


In [ ]:
def bootstrap_sample(X, y):
    # Function for bootstrap sampling. Hint: use np.random.choice for idxs
    idxs = np.random.choice(range(X.shape[0]), X.shape[0])
    X_sample, y_samples = X[idxs], y[idxs]
    return X_sample, y_samples

def most_common_label(y):
    """
      Determine the target label with maximum frequency. Again, you can use collections.Counter()
      y: target
      """
    most_common = np.argmax(np.bincount(y))
    return most_common

In [ ]:
class RandomForest:
    
    def __init__(self, n_trees=10, min_samples_split=2,
                 max_depth=100, num_features=None):
        # Initialize the variabes
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_features = num_features
        self.trees = []

    def fit(self, X, y):
        """
        Create n_tree decision trees using the training data
        X: data
        y: target
        """
        for i in range(self.n_trees):
            X_sample, y_samples = bootstrap_sample(X, y)
            tree = DecisionTree(self.min_samples_split, self.max_depth, self.num_features)
            tree.fit(X_sample, y_samples)
            self.trees.append(tree)

    def predict(self, X):
        """
        Write a predict function to make predictions
        X: data
        Return predictions
        """
        y_pred = []
        for i in range(len(X)):
            preds = []
            for tree in self.trees:
                preds.append(tree.predict(X[i]))
            y_pred.append(most_common_label(preds))
        return y_pred

In [ ]:
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

In [ ]:
# Do not change anything in this cell
import time

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
for n_trees in [1,3,5,10,20]:
    clf = RandomForest(n_trees=n_trees, max_depth=10)
    start = time.time()
    clf.fit(X_train, y_train)
    train_finish = time.time()
    y_pred = clf.predict(X_test)
    test_finish = time.time()
    acc = accuracy(y_test, y_pred)
    print (f"Num of trees : {n_trees} Accuracy: {acc:.2f} Training took {train_finish-start:.2f}s Testing took {test_finish - train_finish:.2f}s")

Num of trees : 1 Accuracy: 0.90 Training took 0.30s Testing took 0.00s
Num of trees : 3 Accuracy: 0.92 Training took 0.93s Testing took 0.00s
Num of trees : 5 Accuracy: 0.89 Training took 1.52s Testing took 0.00s
Num of trees : 10 Accuracy: 0.93 Training took 3.24s Testing took 0.00s
Num of trees : 20 Accuracy: 0.95 Training took 6.16s Testing took 0.01s
